# Import libraries

In [11]:
import subprocess
from pathlib import Path
import pycolmap
import os
import shutil


# Functions

In [12]:
def clean_colmap(base_dir="assets"):
    """
    Recursively delete all COLMAP-generated files such as `pycolmap.db` 
    and `pycolmap_output/` directories from the given base directory.

    Parameters:
        base_dir (str): Root directory to start cleanup (default: 'assets')
    """
    for root, dirs, files in os.walk(base_dir):
        # Delete pycolmap.db files
        if "pycolmap.db" in files:
            db_path = os.path.join(root, "pycolmap.db")
            os.remove(db_path)
            print(f"Deleted file: {db_path}")

        # Delete pycolmap_output directories
        if "pycolmap_output" in dirs:
            output_path = os.path.join(root, "pycolmap_output")
            shutil.rmtree(output_path)
            print(f"Deleted directory: {output_path}")

In [13]:
def prepare_database_with_colmap(image_dir):
    image_dir = Path(image_dir)
    db_path = image_dir / "pycolmap.db"
    
    if db_path.exists():
        print("ℹ️ Removing existing database to force fresh extraction and matching.")
        db_path.unlink()
    # 1. Feature extraction
    subprocess.run([
        "colmap", "feature_extractor",
        "--database_path", str(db_path),
        "--image_path", str(image_dir)
    ], check=True)

    # 2. Feature matching
    subprocess.run([
        "colmap", "exhaustive_matcher",
        "--database_path", str(db_path)
    ], check=True)

    return db_path

def run_pycolmap_sfm_after_cli(image_dir):
    image_dir = Path(image_dir)
    db_path = prepare_database_with_colmap(image_dir)
    output_dir = image_dir / "pycolmap_output"
    output_dir.mkdir(exist_ok=True)

    reconstructions = pycolmap.incremental_mapping(
        database_path=str(db_path),
        image_path=str(image_dir),
        output_path=str(output_dir)
    )

    if not reconstructions:
        raise RuntimeError("❌ No reconstruction produced.")
    
    sfm = reconstructions[0]
    print("✅ SfM complete")
    print(f"Images registered: {len(sfm.images)}")
    print(f"3D points: {len(sfm.points3D)}")
    return sfm

In [14]:
clean_colmap()

Deleted file: assets/marine/pycolmap.db
Deleted directory: assets/marine/pycolmap_output


In [15]:
image_dir = "assets/knife/images_clean"
db_path = "assets/knife/pycolmap.db"

subprocess.run([
    "colmap", "feature_extractor",
    "--database_path", db_path,
    "--image_path", image_dir
], check=True)


QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700
I20250715 22:24:39.602499 239964 misc.cc:198] 
Feature extraction
I20250715 22:24:40.363171 239994 feature_extraction.cc:254] Processed file [1/19]
I20250715 22:24:40.363214 239994 feature_extraction.cc:257]   Name:            Screenshot 2025-07-15 124237.png
I20250715 22:24:40.363219 239994 feature_extraction.cc:283]   Dimensions:      2560 x 1600
I20250715 22:24:40.363224 239994 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I20250715 22:24:40.363229 239994 feature_extraction.cc:289]   Focal Length:    3072.00px
I20250715 22:24:40.363236 239994 feature_extraction.cc:302]   Features:        931
I20250715 22:24:40.669579 239994 feature_extraction.cc:254] Processed file [2/19]
I20250715 22:24:40.669634 239994 feature_extraction.cc:257]   Name:            Screenshot 2025-07-15 124253.png
I20250715 22:24:40.669641 239994 feature_extraction.cc:283]   Dimensions:      2560 x 1600
I

CompletedProcess(args=['colmap', 'feature_extractor', '--database_path', 'assets/knife/pycolmap.db', '--image_path', 'assets/knife/images_clean'], returncode=0)

In [16]:
subprocess.run([
    "colmap", "exhaustive_matcher",
    "--database_path", "assets/knife/pycolmap.db"
], check=True)


QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700
I20250715 22:24:51.986747 240012 misc.cc:198] 
Exhaustive feature matching
I20250715 22:24:52.005569 240012 feature_matching.cc:231] Matching block [1/1, 1/1]
I20250715 22:24:53.729394 240012 feature_matching.cc:46]  in 1.724s
I20250715 22:24:53.729863 240012 timer.cc:91] Elapsed time: 0.029 [minutes]


CompletedProcess(args=['colmap', 'exhaustive_matcher', '--database_path', 'assets/knife/pycolmap.db'], returncode=0)

In [17]:
import subprocess
from pathlib import Path

def run_colmap_mapper(image_dir, db_path, output_dir="pycolmap_output"):
    image_dir = Path(image_dir)
    db_path = Path(db_path)
    output_dir = image_dir.parent / output_dir  # e.g., assets/marine/pycolmap_output

    if output_dir.exists():
        print(f"⚠️ Removing existing reconstruction at {output_dir}")
        subprocess.run(["rm", "-rf", str(output_dir)], check=True)

    output_dir.mkdir(parents=True, exist_ok=True)

    subprocess.run([
        "colmap", "mapper",
        "--database_path", str(db_path),
        "--image_path", str(image_dir),
        "--output_path", str(output_dir),
        "--Mapper.num_threads", "8",
        "--Mapper.init_min_tri_angle", "4"  # a bit more robust on noisy data
    ], check=True)

    return output_dir


In [18]:
image_dir = "assets/knife/images_clean"
db_path = "assets/knife/pycolmap.db"
sparse_model_dir = run_colmap_mapper(image_dir, db_path)


I20250715 22:25:03.418073 240049 misc.cc:198] 
Loading database
I20250715 22:25:03.418906 240049 database_cache.cc:54] Loading cameras...
I20250715 22:25:03.418951 240049 database_cache.cc:64]  19 in 0.000s
I20250715 22:25:03.418958 240049 database_cache.cc:72] Loading matches...
I20250715 22:25:03.419200 240049 database_cache.cc:78]  171 in 0.000s
I20250715 22:25:03.419219 240049 database_cache.cc:94] Loading images...
I20250715 22:25:03.419891 240049 database_cache.cc:143]  19 in 0.001s (connected 19)
I20250715 22:25:03.419930 240049 database_cache.cc:154] Building correspondence graph...
I20250715 22:25:03.421002 240049 database_cache.cc:190]  in 0.001s (ignored 0)
I20250715 22:25:03.421037 240049 timer.cc:91] Elapsed time: 0.000 [minutes]
I20250715 22:25:03.421733 240049 misc.cc:198] 
Finding good initial image pair
I20250715 22:25:03.429777 240049 misc.cc:198] 
Initializing with image pair #17 and #16
I20250715 22:25:03.430673 240049 misc.cc:198] 
Global bundle adjustment
I2025071

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.163319e+02    0.00e+00    9.92e+03   0.00e+00   0.00e+00  1.00e+04        0    4.23e-04    1.24e-03
   1  2.061616e+02    1.02e+01    3.44e+02   0.00e+00   1.00e+00  3.00e+04        1    6.68e-04    1.96e-03
   2  2.024234e+02    3.74e+00    2.24e+02   9.00e+01   8.73e-01  5.14e+04        1    5.62e-04    2.54e-03
   3  1.991615e+02    3.26e+00    1.24e+03   1.32e+02   6.08e-01  5.20e+04        1    5.49e-04    3.10e-03
   4  1.949108e+02    4.25e+00    7.65e+02   1.15e+02   8.01e-01  6.65e+04        1    5.98e-04    3.72e-03
   5  1.922844e+02    2.63e+00    1.04e+03   1.25e+02   6.51e-01  6.84e+04        1    5.64e-04    4.29e-03
   6  1.897089e+02    2.58e+00    1.11e+03   1.09e+02   7.32e-01  7.60e+04        1    6.95e-04    5.00e-03
   7  1.880183e+02    1.69e+00    1.50e+03   1.03e+02   6.34e-01  7.75e+04        1    5.82e-04    5.62e-03
   8  1.865642e+02    1.45e+

I20250715 22:25:03.657392 240049 misc.cc:205] 
Bundle adjustment report
------------------------
I20250715 22:25:03.657433 240049 bundle_adjustment.cc:942] 
    Residuals : 880
   Parameters : 602
   Iterations : 101
         Time : 0.040725 [s]
 Initial cost : 0.178261 [px]
   Final cost : 0.100997 [px]
  Termination : No convergence

I20250715 22:25:03.657596 240049 incremental_mapper.cc:175] => Completed observations: 0
I20250715 22:25:03.657626 240049 incremental_mapper.cc:178] => Merged observations: 0
I20250715 22:25:03.657655 240049 incremental_mapper.cc:160] => Filtered observations: 0
I20250715 22:25:03.657658 240049 incremental_mapper.cc:119] => Changed observations: 0.000000
I20250715 22:25:03.657662 240049 incremental_mapper.cc:167] => Filtered images: 0
I20250715 22:25:03.690469 240049 misc.cc:198] 
Registering image #16 (4)
I20250715 22:25:03.690511 240049 incremental_mapper.cc:495] => Image sees 55 / 349 points
I20250715 22:25:03.695258 240049 misc.cc:205] 
Pose refineme

  67  1.122054e+01    2.78e-07    2.04e+00   3.69e-01   8.94e-01  1.40e+08        1    4.61e-04    3.12e-02
  68  1.122054e+01    7.80e-08    7.28e-01   2.20e-01   9.56e-01  4.21e+08        1    4.47e-04    3.17e-02
  69  1.122054e+01    1.03e-08    9.92e-02   7.45e-02   1.07e+00  1.26e+09        1    4.52e-04    3.21e-02
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.008439e+02    0.00e+00    3.73e+03   0.00e+00   0.00e+00  1.00e+04        0    3.64e-04    7.33e-04
   1  9.190924e+01    8.93e+00    1.65e+03   0.00e+00   9.93e-01  3.00e+04        1    6.02e-04    1.35e-03
   2  9.096978e+01    9.39e-01    3.99e+03   1.81e+01   9.35e-01  8.74e+04        1    6.11e-04    1.97e-03
   3  9.072058e+01    2.49e-01    1.32e+04   2.81e+01   2.27e-01  7.52e+04        1    5.60e-04    2.54e-03
   4  8.966150e+01    1.06e+00    1.18e+04   2.16e+01   7.06e-01  8.08e+04        1    5.78e-04    3.13e-03
   5  8.928981e+01    3.72e-

I20250715 22:25:03.884132 240049 misc.cc:198] 
Registering image #17 (5)
I20250715 22:25:03.884177 240049 incremental_mapper.cc:495] => Image sees 62 / 353 points
I20250715 22:25:03.889472 240049 misc.cc:205] 
Pose refinement report
----------------------
I20250715 22:25:03.889498 240049 bundle_adjustment.cc:942] 
    Residuals : 122
   Parameters : 8
   Iterations : 12
         Time : 0.000562906 [s]
 Initial cost : 0.724074 [px]
   Final cost : 0.296141 [px]
  Termination : Convergence

I20250715 22:25:03.889551 240049 incremental_mapper.cc:40] => Continued observations: 61
I20250715 22:25:03.889683 240049 incremental_mapper.cc:43] => Added observations: 17
I20250715 22:25:03.903618 240049 misc.cc:205] 
Bundle adjustment report
------------------------
I20250715 22:25:03.903640 240049 bundle_adjustment.cc:942] 
    Residuals : 1152
   Parameters : 645
   Iterations : 26
         Time : 0.01367 [s]
 Initial cost : 0.405914 [px]
   Final cost : 0.318628 [px]
  Termination : No converge

...

I20250715 22:25:06.681380 240049 misc.cc:205] 
Bundle adjustment report
------------------------
I20250715 22:25:06.681427 240049 bundle_adjustment.cc:942] 
    Residuals : 2278
   Parameters : 558
   Iterations : 51
         Time : 0.112532 [s]
 Initial cost : 0.20295 [px]
   Final cost : 0.202637 [px]
  Termination : No convergence

I20250715 22:25:06.681762 240049 incremental_mapper.cc:175] => Completed observations: 0
I20250715 22:25:06.681860 240049 incremental_mapper.cc:178] => Merged observations: 0
I20250715 22:25:06.681917 240049 incremental_mapper.cc:160] => Filtered observations: 0
I20250715 22:25:06.681931 240049 incremental_mapper.cc:119] => Changed observations: 0.000000
I20250715 22:25:06.681938 240049 incremental_mapper.cc:167] => Filtered images: 0
I20250715 22:25:06.718531 240049 misc.cc:198] 
Registering image #10 (18)
I20250715 22:25:06.718580 240049 incremental_mapper.cc:495] => Image sees 56 / 100 points
I20250715 22:25:06.729302 240049 misc.cc:205] 
Pose refineme

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  9.618786e+01    0.00e+00    1.89e+03   0.00e+00   0.00e+00  1.00e+04        0    1.02e-03    1.86e-03
   1  9.531963e+01    8.68e-01    1.04e+03   0.00e+00   9.59e-01  3.00e+04        1    2.66e-03    4.54e-03
   2  9.499204e+01    3.28e-01    1.26e+03   1.03e+01   8.58e-01  4.74e+04        1    2.53e-03    7.09e-03
   3  9.480893e+01    1.83e-01    6.59e+02   1.73e+01   9.50e-01  1.42e+05        1    2.53e-03    9.62e-03
   4  9.471612e+01    9.28e-02    6.22e+02   3.09e+01   9.79e-01  4.27e+05        1    2.60e-03    1.22e-02
   5  9.466039e+01    5.57e-02    4.99e+02   3.37e+01   9.74e-01  1.28e+06        1    2.60e-03    1.49e-02
   6  9.461437e+01    4.60e-02    6.45e+02   3.89e+01   8.76e-01  2.23e+06        1    2.57e-03    1.75e-02
   7  9.457275e+01    4.16e-02    9.70e+02   9.98e+01   8.87e-01  4.15e+06        1    2.52e-03    2.00e-02
   8  9.454234e+01    3.04e-

I20250715 22:25:06.937906 240049 misc.cc:205] 
Bundle adjustment report
------------------------
I20250715 22:25:06.937949 240049 bundle_adjustment.cc:942] 
    Residuals : 2516
   Parameters : 586
   Iterations : 51
         Time : 0.127261 [s]
 Initial cost : 0.195526 [px]
   Final cost : 0.1935 [px]
  Termination : No convergence

I20250715 22:25:06.938253 240049 incremental_mapper.cc:175] => Completed observations: 0
I20250715 22:25:06.938352 240049 incremental_mapper.cc:178] => Merged observations: 0
I20250715 22:25:06.938416 240049 incremental_mapper.cc:160] => Filtered observations: 7
I20250715 22:25:06.938429 240049 incremental_mapper.cc:119] => Changed observations: 0.005564
I20250715 22:25:06.938437 240049 misc.cc:198] 
Global bundle adjustment
I20250715 22:25:07.068604 240049 misc.cc:205] 
Bundle adjustment report
------------------------
I20250715 22:25:07.068645 240049 bundle_adjustment.cc:942] 
    Residuals : 2488
   Parameters : 565
   Iterations : 51
         Time : 0.

  32  9.388666e+01    2.80e-03    1.94e+03   3.03e+02   4.07e-01  7.09e+06        1    2.72e-03    8.30e-02
  33  9.388390e+01    2.77e-03    1.93e+03   3.01e+02   4.08e-01  7.05e+06        1    2.54e-03    8.56e-02
  34  9.388116e+01    2.74e-03    1.92e+03   2.98e+02   4.09e-01  7.01e+06        1    2.53e-03    8.81e-02
  35  9.387845e+01    2.71e-03    1.91e+03   2.96e+02   4.09e-01  6.96e+06        1    2.51e-03    9.07e-02
  36  9.387578e+01    2.68e-03    1.91e+03   2.93e+02   4.10e-01  6.92e+06        1    2.54e-03    9.32e-02
  37  9.387313e+01    2.65e-03    1.90e+03   2.91e+02   4.11e-01  6.88e+06        1    2.68e-03    9.59e-02
  38  9.387051e+01    2.62e-03    1.89e+03   2.88e+02   4.12e-01  6.85e+06        1    2.61e-03    9.86e-02
  39  9.386792e+01    2.59e-03    1.88e+03   2.86e+02   4.13e-01  6.81e+06        1    2.78e-03    1.01e-01
  40  9.386535e+01    2.57e-03    1.87e+03   2.84e+02   4.14e-01  6.78e+06        1    2.56e-03    1.04e-01
  41  9.386280e+01    2.55e-

I20250715 22:25:07.196198 240049 misc.cc:205] 
Bundle adjustment report
------------------------
I20250715 22:25:07.196241 240049 bundle_adjustment.cc:942] 
    Residuals : 2468
   Parameters : 550
   Iterations : 51
         Time : 0.126494 [s]
 Initial cost : 0.194926 [px]
   Final cost : 0.194831 [px]
  Termination : No convergence

I20250715 22:25:07.196573 240049 incremental_mapper.cc:175] => Completed observations: 0
I20250715 22:25:07.196669 240049 incremental_mapper.cc:178] => Merged observations: 0
I20250715 22:25:07.196715 240049 incremental_mapper.cc:160] => Filtered observations: 0
I20250715 22:25:07.196718 240049 incremental_mapper.cc:119] => Changed observations: 0.000000
I20250715 22:25:07.196723 240049 incremental_mapper.cc:167] => Filtered images: 0
I20250715 22:25:07.236241 240049 timer.cc:91] Elapsed time: 0.064 [minutes]


In [19]:
def convert_model_to_ply(model_dir):
    ply_path = model_dir / "sparse.ply"

    subprocess.run([
        "colmap", "model_converter",
        "--input_path", str(model_dir),
        "--output_path", str(ply_path),
        "--output_type", "PLY"
    ], check=True)

    print(f"✅ Exported sparse model to: {ply_path}")
    return ply_path


In [20]:
ply_model = convert_model_to_ply(sparse_model_dir / "0")  # '0' is the first reconstruction

✅ Exported sparse model to: assets/knife/pycolmap_output/0/sparse.ply
